In [35]:
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
from selenium import webdriver
import time
import pandas as pd
import os
import sys

In [46]:
path = "/Users/yingli/Documents/git_project/ds_salary_proj/chromedriver"
slp_time = 15

In [63]:
def get_jobs(keyword, num_jobs, verbose, path, slp_time):
    
    '''Gathers jobs as a dataframe, scraped from Glassdoor'''
    
    #Initializing the webdriver
    options = webdriver.ChromeOptions()
    
    #Uncomment the line below if you'd like to scrape without a new Chrome window every time.
    #options.add_argument('headless')
    
    #Change the path to where chromedriver is in your home folder.
    driver = webdriver.Chrome(executable_path=path, options=options)
    driver.set_window_size(1120, 1000)

    url = 'https://www.glassdoor.com/Job/jobs.htm?suggestCount=0&suggestChosen=false&clickSource=searchBtn&typedKeyword=' + keyword +'&locT=&locId=&jobType=&context=Jobs&sc.keyword='+ keyword +'&dropdown=0' 
#    url = 'https://www.glassdoor.com/Job/jobs.htm?sc.keyword="' + keyword + '"&locT=C&locId=1147401&locKeyword=San%20Francisco,%20CA&jobType=all&fromAge=-1&minSalary=0&includeNoSalaryJobs=true&radius=100&cityId=-1&minRating=0.0&industryId=-1&sgocId=-1&seniorityType=all&companyId=-1&employerSizes=0&applicationType=0&remoteWorkType=0'
    driver.get(url)
    jobs = []

    while len(jobs) < num_jobs:  #If true, should be still looking for new jobs.

        #Let the page load. Change this number based on your internet speed.
        #Or, wait until the webpage is loaded, instead of hardcoding it.
        time.sleep(slp_time)

        #Test for the "Sign Up" prompt and get rid of it.
        try:
            driver.find_element_by_class_name("selected").click()
        except ElementClickInterceptedException:
            pass

        time.sleep(.1)

        try:
 #           driver.find_element_by_class_name("ModalStyle__xBtn___29PT9").click()  #clicking to the X.
            driver.find_element_by_css_selector('[alt="Close"]').click() 
            print(' X out worked')
        except NoSuchElementException:
            print(' X out failed')
            pass

        
        #Going through each job in this page
        job_buttons = driver.find_elements_by_class_name("jl")  #jl for Job Listing. These are the buttons we're going to click.
        for job_button in job_buttons:  

            print("Progress: {}".format("" + str(len(jobs)) + "/" + str(num_jobs)))
            if len(jobs) >= num_jobs:
                break

            job_button.click()  #You might 
            time.sleep(1)
            collected_successfully = False
            
            while not collected_successfully:
                try:
                    company_name = driver.find_element_by_xpath('.//div[@class="employerName"]').text
                    location = driver.find_element_by_xpath('.//div[@class="location"]').text
                    job_title = driver.find_element_by_xpath('.//div[contains(@class, "title")]').text
                    job_description = driver.find_element_by_xpath('.//div[@class="jobDescriptionContent desc"]').text
                    collected_successfully = True
                except:
                    time.sleep(5)

            try:
                salary_estimate = driver.find_element_by_xpath('.//span[@class="css-1uyte9r css-hca4ks e1wijj242"]').text
            except NoSuchElementException:
                salary_estimate = -1 #You need to set a "not found value. It's important."
            
            try:
                rating = driver.find_element_by_xpath('.//span[@class="rating"]').text
            except NoSuchElementException:
                rating = -1 #You need to set a "not found value. It's important."

            #Printing for debugging
            if verbose:
                print("Job Title: {}".format(job_title))
                print("Salary Estimate: {}".format(salary_estimate))
                print("Job Description: {}".format(job_description[:500]))
                print("Rating: {}".format(rating))
                print("Company Name: {}".format(company_name))
                print("Location: {}".format(location))

            #Going to the Company tab...
            #clicking on this:
            #<div class="tab" data-tab-type="overview"><span>Company</span></div>
            try:
                driver.find_element_by_xpath('.//div[@class="tab" and @data-tab-type="overview"]').click()

                try:
                    #<div class="infoEntity">
                    #    <label>Headquarters</label>
                    #    <span class="value">San Francisco, CA</span>
                    #</div>
                    headquarters = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Headquarters"]//following-sibling::*').text
                except NoSuchElementException:
                    headquarters = -1

                try:
                    size = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Size"]//following-sibling::*').text
                except NoSuchElementException:
                    size = -1

                try:
                    founded = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Founded"]//following-sibling::*').text
                except NoSuchElementException:
                    founded = -1

                try:
                    type_of_ownership = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Type"]//following-sibling::*').text
                except NoSuchElementException:
                    type_of_ownership = -1

                try:
                    industry = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Industry"]//following-sibling::*').text
                except NoSuchElementException:
                    industry = -1

                try:
                    sector = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Sector"]//following-sibling::*').text
                except NoSuchElementException:
                    sector = -1

                try:
                    revenue = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Revenue"]//following-sibling::*').text
                except NoSuchElementException:
                    revenue = -1

                try:
                    competitors = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Competitors"]//following-sibling::*').text
                except NoSuchElementException:
                    competitors = -1

            except NoSuchElementException:  #Rarely, some job postings do not have the "Company" tab.
                headquarters = -1
                size = -1
                founded = -1
                type_of_ownership = -1
                industry = -1
                sector = -1
                revenue = -1
                competitors = -1

                
            if verbose:
                print("Headquarters: {}".format(headquarters))
                print("Size: {}".format(size))
                print("Founded: {}".format(founded))
                print("Type of Ownership: {}".format(type_of_ownership))
                print("Industry: {}".format(industry))
                print("Sector: {}".format(sector))
                print("Revenue: {}".format(revenue))
                print("Competitors: {}".format(competitors))
                print("@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@")

            jobs.append({"Job Title" : job_title,
            "Salary Estimate" : salary_estimate,
            "Job Description" : job_description,
            "Rating" : rating,
            "Company Name" : company_name,
            "Location" : location,
            "Headquarters" : headquarters,
            "Size" : size,
            "Founded" : founded,
            "Type of ownership" : type_of_ownership,
            "Industry" : industry,
            "Sector" : sector,
            "Revenue" : revenue,
            "Competitors" : competitors})
            #add job to jobs

        #Clicking on the "next page" button
        try:
            driver.find_element_by_xpath('.//li[@class="next"]//a').click()
        except NoSuchElementException:
            print("Scraping terminated before reaching target number of jobs. Needed {}, got {}.".format(num_jobs, len(jobs)))
            break

    return pd.DataFrame(jobs)  #This line converts the dictionary object into a pandas DataFrame.

In [66]:
#This line will open a new chrome window and start the scraping.
df = get_jobs("data scientist", 25, False, path, slp_time)
df

 X out worked
Progress: 0/25
Progress: 1/25
Progress: 2/25
Progress: 3/25
Progress: 4/25
Progress: 5/25
Progress: 6/25
Progress: 7/25
Progress: 8/25
Progress: 9/25
Progress: 10/25
Progress: 11/25
Progress: 12/25
Progress: 13/25
Progress: 14/25
Progress: 15/25
Progress: 16/25
Progress: 17/25
Progress: 18/25
Progress: 19/25
Progress: 20/25
Progress: 21/25
Progress: 22/25
Progress: 23/25
Progress: 24/25
Progress: 25/25


,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors
0,Staff Data Scientist I,$127K-$203K (Glassdoor est.),Data Scientist – Machine Learning & Real-Time ...,3.4,Valassis\n3.4,"Morrisville, NC",-1,1001 to 5000 Employees,1970,Company - Private,Advertising & Marketing,Business Services,Unknown / Non-Applicable,-1
1,Data Scientist,$75K-$131K (Glassdoor est.),Overview\n\n\nData Scientist\n\nLocations US-M...,3.8,Analysis Group\n3.8,"Boston, MA",-1,1001 to 5000 Employees,1981,Private Practice / Firm,Consulting,Business Services,$100 to $500 million (USD),-1
2,Principal Scientist,-1,Myeloid Therapeutics *\nRole: *Principal Scien...,-1,Myeloid Therapeutics,"Cambridge, MA",-1,-1,-1,-1,-1,-1,-1,-1
3,Deep Learning Algorithm Scientist,-1,VIDA is a growing company with BIG focus. Our ...,3.6,VIDA\n3.6,"Minneapolis, MN",-1,Unknown,-1,Unknown,-1,-1,Less than $1 million (USD),-1
4,Data Analyst Specialist,-1,DATA ANALYST*\nFast growing Columbia based E-C...,-1,Taylor Toolworks LLC,"Columbia, MO",-1,1 to 50 Employees,-1,Company - Private,Other Retail Stores,Retail,Less than $1 million (USD),-1
5,Data Scientist,$80K-$111K (Glassdoor est.),Job Brief\n\nThe ideal candidate will have pre...,2.7,IFG Companies\n2.7,"Atlanta, GA",-1,201 to 500 Employees,1985,Company - Private,Insurance Carriers,Insurance,Unknown / Non-Applicable,-1
6,Certified Lab Scientist or Certified Lab Tech,-1,Prairie Ridge Health is looking for a team mem...,-1,Prairie Ridge Health,"Columbus, WI",-1,201 to 500 Employees,-1,Hospital,-1,-1,Less than $1 million (USD),-1
7,Senior Data Scientist Virtual Hiring Event,$84K-$136K (Glassdoor est.),Plymouth Rock Assurance Virtual Hiring Event!\...,3.6,Plymouth Rock Assurance\n3.6,"Horsham, PA",-1,1001 to 5000 Employees,1982,Company - Private,Insurance Carriers,Insurance,$10 to $25 million (USD),-1
8,Data Scientist,$55K-$93K (Glassdoor est.),"Sartorius Stedim Data Analytics, a market lead...",3.6,Sartorius\n3.6,"New Oxford, PA",-1,5001 to 10000 Employees,1870,Company - Public,Biotech & Pharmaceuticals,Biotech & Pharmaceuticals,$1 to $2 billion (USD),-1
9,Data Scientist,$58K-$95K (Glassdoor est.),STRATACACHE provides salable customer experien...,3.6,STRATACACHE\n3.6,"Dayton, OH",-1,201 to 500 Employees,1999,Company - Private,Computer Hardware & Software,Information Technology,$100 to $500 million (USD),-1
